In [2]:
import sys
from pathlib import Path
import pandas as pd

sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
from cogwheel import data, gw_utils, gw_plotting, utils
from dot_pe import inference


/Users/jonatahm/miniconda3/envs/gwias/lib/python3.11/site-packages/gwpy/time/__init__.py:36: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  from lal import LIGOTimeGPS


# Generate synthetic data for inference

In [12]:
# create data
event_data_kwargs = {
    "detector_names": "HLV",
    "duration": 120.0,
    "asd_funcs": ["asd_H_O3", "asd_L_O3", "asd_V_O3"],
    "tgps": 0.0,
    "fmax": 1600.0,
}


event_data = data.EventData.gaussian_noise(
    eventname="example_eventdata", **event_data_kwargs, seed=20250311
)

mchirp = 75
q = 1 / 2

m1, m2 = gw_utils.mchirpeta_to_m1m2(mchirp, gw_utils.q_to_eta(q))
injection_par_dic = dict(
    m1=m1,
    m2=m2,
    ra=0.5,
    dec=0.5,
    iota=np.pi * 1 / 3,
    psi=1.0,
    phi_ref=12.0,
    s1z=0.6,
    s2z=0.6,
    s1x_n=0.1,
    s1y_n=0.2,
    s2x_n=0.3,
    s2y_n=-0.2,
    l1=0.0,
    l2=0.0,
    tgps=0.0,
    f_ref=50.0,
    d_luminosity=5e3,
    t_geocenter=0.0,
)

event_data.inject_signal(injection_par_dic, "IMRPhenomXODE")

print(event_data.injection["d_h"] - event_data.injection["h_h"] / 2)
print(sum(event_data.injection["d_h"] - event_data.injection["h_h"] / 2))

[13.05668831 32.44716761  1.04042227]
46.544278189095614


In [11]:
# Alternatively, load data form file
event_data = data.EventData.from_npz(filename="example_eventdata.npz")

FileNotFoundError: [Errno 2] No such file or directory: 'example_eventdata.npz'

Set bank folder, in the relevant mass range.

See how to create mass ranges in create_sample_bank.ipynb


In [ ]:
bank_folder = "test_bank"
bank_folder = Path(bank_folder)
event_dir = event_data.eventname

In [ ]:
rundir = inference.run(
    event_dir=event_dir,
    event=event_data,
    bank_folder=bank_folder,
    n_int=2**14,
    n_ext=128,
    n_phi=32,
    n_t=64,
    i_int_start=0,
    blocksize=2**10,
    single_detector_blocksize=2**10,
    seed=42,
    size_limit=10**6,
    draw_subset=True,
    n_draws=None,
)

In [ ]:
rundirs = sorted(
    Path("example_eventdata").glob("run_*"),
    key=lambda x: int(x.name.split("_")[-1]),
)

rundir = rundirs[-1]
summary_results = utils.read_json(rundirs[-1] / "summary_results.json")

for k, v in summary_results.items():
    print(k, v)
    

In [ ]:
samples = pd.read_feather(rundirs[-1] / "samples.feather")
params = ["mchirp", "lnq", "chieff", "cumchidiff", "costheta_jn", "lnl"]
gw_plotting.CornerPlot(samples, params=params, smooth=1).plot()

In [ ]:
samples